## Module 2 : Spatialisation de la Sécheresse par Krigeage (45 min)

### 2.1 Préparation des données pour interpolation

In [ ]:
# Sélection des données pour juillet 2016 (pic de sécheresse)
target_date = '2016-07-01'
spi_july_2016 = drought_data_spi[
    drought_data_spi['date'] == target_date
].copy()

print(f"📅 DONNÉES POUR SPATIALISATION - {target_date}:")
print("="*60)
print(f"{'Station':<15} {'Latitude':<10} {'Longitude':<11} {'SPI-3':<8}")
print("="*60)

for _, row in spi_july_2016.iterrows():
    print(f"{row['station']:<15} {row['latitude']:<10.3f} {row['longitude']:<11.3f} {row['spi_3']:<8.2f}")

# Vérification des données manquantes
missing_spi = spi_july_2016['spi_3'].isna().sum()
if missing_spi > 0:
    print(f"\n⚠️ ATTENTION: {missing_spi} stations avec SPI manquant")
    spi_july_2016 = spi_july_2016.dropna(subset=['spi_3'])
    print(f"Stations utilisables: {len(spi_july_2016)}")
else:
    print(f"\n✅ Toutes les stations ont des valeurs SPI valides")

# Préparation des coordonnées pour krigeage
coords = spi_july_2016[['longitude', 'latitude']].values
spi_values = spi_july_2016['spi_3'].values

print(f"\n📊 STATISTIQUES SPI JUILLET 2016:")
print(f"Minimum: {spi_values.min():.2f}")
print(f"Maximum: {spi_values.max():.2f}")
print(f"Moyenne: {spi_values.mean():.2f}")
print(f"Écart-type: {spi_values.std():.2f}")

### 2.2 Création de la grille d'interpolation Nord CI

In [ ]:
# Définition des limites géographiques du Nord CI
north_ci_bounds = {
    'min_lat': 7.5,    # Sud (limite zone soudanienne)
    'max_lat': 10.5,   # Nord (frontière Mali/Burkina)
    'min_lon': -7.5,   # Ouest (frontière Guinée/Mali)
    'max_lon': -2.5    # Est (frontière Ghana/Burkina)
}

# Création grille d'interpolation (résolution 10km)
resolution_km = 10
centroids_north = get_grid_points(**north_ci_bounds, res_km=resolution_km)

print(f"🗺️ GRILLE D'INTERPOLATION NORD CI:")
print(f"Résolution: {resolution_km} km x {resolution_km} km")
print(f"Nombre de points: {len(centroids_north):,}")
print(f"Étendue latitude: {north_ci_bounds['min_lat']:.1f}° à {north_ci_bounds['max_lat']:.1f}°N")
print(f"Étendue longitude: {north_ci_bounds['min_lon']:.1f}° à {north_ci_bounds['max_lon']:.1f}°W")

# Calcul de la superficie couverte
lat_range = north_ci_bounds['max_lat'] - north_ci_bounds['min_lat']
lon_range = north_ci_bounds['max_lon'] - north_ci_bounds['min_lon']
area_deg2 = lat_range * lon_range
area_km2 = area_deg2 * 111 * 111  # Approximation 1° ≈ 111 km

print(f"Superficie approximative: {area_km2:,.0f} km²")

# Visualisation des stations et de la grille
plt.figure(figsize=(12, 8))

# Points de grille (échantillonnage pour lisibilité)
sample_grid = centroids_north.iloc[::20]  # 1 point sur 20
plt.scatter(sample_grid['lon'], sample_grid['lat'], 
           c='lightgray', s=1, alpha=0.5, label='Grille interpolation')

# Stations météo avec valeurs SPI
scatter = plt.scatter(spi_july_2016['longitude'], spi_july_2016['latitude'],
                     c=spi_july_2016['spi_3'], cmap='RdYlBu', 
                     s=200, edgecolors='black', linewidth=2,
                     vmin=-2.5, vmax=0.5, label='Stations météo')

# Annotations des stations
for _, row in spi_july_2016.iterrows():
    plt.annotate(f"{row['station']}\n({row['spi_3']:.1f})", 
                xy=(row['longitude'], row['latitude']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=9, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

plt.colorbar(scatter, label='SPI-3 Juillet 2016')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Stations Météo et Grille d\'Interpolation - Nord Côte d\'Ivoire', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2.3 Interpolation par krigeage gaussien

In [ ]:
# Configuration du modèle de krigeage
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern, WhiteKernel
from sklearn.model_selection import cross_val_score

# Test de différents kernels
kernels = {
    'RBF': RBF(length_scale=1.0, length_scale_bounds=(0.1, 10.0)),
    'Matern_1.5': Matern(length_scale=1.0, nu=1.5),
    'Matern_2.5': Matern(length_scale=1.0, nu=2.5),
    'RBF_Noise': RBF(length_scale=1.0) + WhiteKernel(noise_level=0.1)
}

print("🔍 SÉLECTION DU MEILLEUR KERNEL (validation croisée):")
print("="*60)
print(f"{'Kernel':<15} {'Score CV':<12} {'Std':<8}")
print("="*60)

best_score = -np.inf
best_kernel = None
best_gp = None

for kernel_name, kernel in kernels.items():
    # Modèle de krigeage
    gp = GaussianProcessRegressor(kernel=kernel, 
                                 alpha=1e-6,  # Régularisation
                                 normalize_y=True,
                                 random_state=42)
    
    # Validation croisée (Leave-One-Out pour petit échantillon)
    cv_scores = cross_val_score(gp, coords, spi_values, 
                               cv=len(coords), scoring='neg_mean_squared_error')
    
    mean_score = cv_scores.mean()
    std_score = cv_scores.std()
    
    print(f"{kernel_name:<15} {mean_score:<12.3f} {std_score:<8.3f}")
    
    if mean_score > best_score:
        best_score = mean_score
        best_kernel = kernel_name
        best_gp = gp

print(f"\n🏆 MEILLEUR KERNEL: {best_kernel} (Score: {best_score:.3f})")

# Ajustement du modèle final
print("\n⚙️ Ajustement du modèle de krigeage...")
best_gp.fit(coords, spi_values)

# Prédiction sur la grille
grid_coords = centroids_north[['lon', 'lat']].values
spi_pred, spi_std = best_gp.predict(grid_coords, return_std=True)

# Ajout des résultats à la grille
centroids_north['spi_3_pred'] = spi_pred
centroids_north['spi_3_std'] = spi_std

print(f"✅ Interpolation terminée!")
print(f"SPI prédit - Min: {spi_pred.min():.2f}, Max: {spi_pred.max():.2f}, Moyenne: {spi_pred.mean():.2f}")
print(f"Incertitude - Min: {spi_std.min():.2f}, Max: {spi_std.max():.2f}, Moyenne: {spi_std.mean():.2f}")

# Classification de la sécheresse sur la grille
centroids_north['drought_category'] = centroids_north['spi_3_pred'].apply(classify_drought)

# Statistiques par catégorie
print(f"\n📊 RÉPARTITION DE LA SÉCHERESSE (% de la zone):")
drought_stats = centroids_north['drought_category'].value_counts(normalize=True) * 100
for category, percent in drought_stats.items():
    print(f"  {category}: {percent:.1f}%")

### 2.4 Visualisation de la spatialisation

In [ ]:
# Création de cartes de la sécheresse spatialisée
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Carte 1: SPI interpolé
ax1 = axes[0]
scatter1 = ax1.scatter(centroids_north['lon'], centroids_north['lat'],
                      c=centroids_north['spi_3_pred'], cmap='RdYlBu',
                      s=15, alpha=0.8, vmin=-2.5, vmax=0.5)

# Superposition des stations
ax1.scatter(spi_july_2016['longitude'], spi_july_2016['latitude'],
           c=spi_july_2016['spi_3'], cmap='RdYlBu',
           s=300, edgecolors='black', linewidth=3,
           vmin=-2.5, vmax=0.5, marker='s')

# Annotations stations
for _, row in spi_july_2016.iterrows():
    ax1.annotate(row['station'], 
                xy=(row['longitude'], row['latitude']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.9))

cbar1 = plt.colorbar(scatter1, ax=ax1)
cbar1.set_label('SPI-3 Interpolé', fontsize=12)
ax1.set_xlabel('Longitude')
ax1.set_ylabel('Latitude')
ax1.set_title('SPI-3 Spatialisé - Juillet 2016\n(Carrés = Stations, Points = Interpolation)', fontweight='bold')
ax1.grid(True, alpha=0.3)

# Carte 2: Incertitude de l'interpolation
ax2 = axes[1]
scatter2 = ax2.scatter(centroids_north['lon'], centroids_north['lat'],
                      c=centroids_north['spi_3_std'], cmap='Reds',
                      s=15, alpha=0.8)

# Stations (incertitude nulle)
ax2.scatter(spi_july_2016['longitude'], spi_july_2016['latitude'],
           c='blue', s=300, edgecolors='black', linewidth=3, marker='s')

cbar2 = plt.colorbar(scatter2, ax=ax2)
cbar2.set_label('Écart-type SPI', fontsize=12)
ax2.set_xlabel('Longitude')
ax2.set_ylabel('Latitude')
ax2.set_title('Incertitude de l\'Interpolation\n(Bleu = Stations observées)', fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Carte interactive avec Folium
def create_drought_map(centroids_df, stations_df):
    """Crée une carte interactive de la sécheresse"""
    # Centre de la carte
    center_lat = centroids_df['lat'].mean()
    center_lon = centroids_df['lon'].mean()
    
    # Carte de base
    m = folium.Map(location=[center_lat, center_lon], 
                   zoom_start=8, tiles='OpenStreetMap')
    
    # Échantillonnage pour performance
    sample_data = centroids_df.iloc[::5].copy()
    
    # Couleurs selon SPI
    def get_color(spi_val):
        if spi_val >= 0:
            return 'blue'
        elif spi_val >= -1:
            return 'yellow'
        elif spi_val >= -1.5:
            return 'orange'
        elif spi_val >= -2:
            return 'red'
        else:
            return 'darkred'
    
    # Points interpolés
    for _, row in sample_data.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=3,
            popup=f"SPI-3: {row['spi_3_pred']:.2f}<br>Catégorie: {row['drought_category']}",
            color=get_color(row['spi_3_pred']),
            fillColor=get_color(row['spi_3_pred']),
            fillOpacity=0.7
        ).add_to(m)
    
    # Stations météo
    for _, row in stations_df.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=f"<b>{row['station']}</b><br>SPI-3: {row['spi_3']:.2f}<br>Catégorie: {classify_drought(row['spi_3'])}",
            icon=folium.Icon(color='black', icon='info-sign')
        ).add_to(m)
    
    return m

# Création de la carte interactive
drought_map = create_drought_map(centroids_north, spi_july_2016)
drought_map.save('results/carte_secheresse_juillet_2016.html')
print("🗺️ Carte interactive sauvegardée: results/carte_secheresse_juillet_2016.html")

# Affichage de la carte
drought_map

## Module 3 : Création de l'Aléa Sécheresse CLIMADA (30 min)

### 3.1 Transformation SPI en aléa CLIMADA

In [ ]:
# Création de l'aléa sécheresse CLIMADA
hazard_drought = Hazard('DR')  # DR = Drought

# Configuration des centroids
hazard_drought.centroids.set_lat_lon(centroids_north['lat'].values, 
                                     centroids_north['lon'].values)

# Transformation SPI en intensité d'aléa
# Convention: intensité = -SPI (plus SPI est négatif, plus l'intensité est forte)
intensity_values = -centroids_north['spi_3_pred'].values
intensity_values = np.maximum(intensity_values, 0)  # Seulement valeurs positives

# Matrice d'intensité (1 événement)
intensity_matrix = intensity_values.reshape(1, -1)
hazard_drought.intensity = intensity_matrix

# Configuration des événements
hazard_drought.event_id = np.array([1])
hazard_drought.event_name = ['Secheresse_Nord_CI_Juillet2016']
hazard_drought.date = np.array([20160701])  # Format YYYYMMDD

# Fréquence basée sur l'analyse historique (estimation)
# Sécheresse sévère ≈ période de retour 10 ans
hazard_drought.frequency = np.array([0.1])  # 1/10 ans

# Matrice de fraction (tous les centroids affectés)
hazard_drought.fraction = np.ones_like(intensity_matrix)

# Vérification de la cohérence
hazard_drought.check()

print("✅ ALÉA SÉCHERESSE CRÉÉ AVEC SUCCÈS!")
print(f"Nombre de centroids: {hazard_drought.centroids.size}")
print(f"Nombre d'événements: {hazard_drought.size[0]}")
print(f"Intensité min: {hazard_drought.intensity.min():.3f}")
print(f"Intensité max: {hazard_drought.intensity.max():.3f}")
print(f"Intensité moyenne: {hazard_drought.intensity.mean():.3f}")
print(f"Fréquence événement: {hazard_drought.frequency[0]:.3f} /an (T={1/hazard_drought.frequency[0]:.0f} ans)")

# Statistiques par niveau d'intensité
print(f"\n📊 RÉPARTITION DES INTENSITÉS:")
intensity_flat = intensity_values[intensity_values > 0]
if len(intensity_flat) > 0:
    print(f"Zones affectées: {len(intensity_flat):,} / {len(intensity_values):,} ({len(intensity_flat)/len(intensity_values)*100:.1f}%)")
    print(f"Intensité moyenne (zones affectées): {intensity_flat.mean():.3f}")
    
    # Classification par seuils
    moderate = np.sum((intensity_flat >= 1) & (intensity_flat < 1.5))
    severe = np.sum((intensity_flat >= 1.5) & (intensity_flat < 2))
    extreme = np.sum(intensity_flat >= 2)
    
    print(f"\n🌡️ RÉPARTITION PAR SÉVÉRITÉ:")
    print(f"  Sécheresse modérée (1.0-1.5): {moderate:,} points ({moderate/len(intensity_flat)*100:.1f}%)")
    print(f"  Sécheresse sévère (1.5-2.0): {severe:,} points ({severe/len(intensity_flat)*100:.1f}%)")
    print(f"  Sécheresse extrême (>2.0): {extreme:,} points ({extreme/len(intensity_flat)*100:.1f}%)")
else:
    print("Aucune zone affectée par la sécheresse")